# Brown Marmorated Stink Bug (BMSB) Risk Analysis in Minnesota: ETL and Data Preparation

Contributors: Mattie Gisselbeck and Luke Zaruba

Description: This notebook implements an automated ETL (Extract, Transform, Load) pipeline to prepare and clean data for analysis.

Last Updated: March 23, 2024

Note: Prior to executing this notebook, it is necessary to execute the <b>extractdata_MnGeo.ps1</b> script to download required datasets which can be found at /bmsb-risk-analysis-minnesota/tools/extractdata_MnGeo.ps1.

### 0. Import Packages and Modules

In [7]:
# Import Packages and Modules
import os
import sys
import arcgis
import arcpy
import pandas as pd

# Import Modules
sys.path.append('/arcgis/home/functions')
from etl import WeatherLoader, ObservationLoader

### 1. Configure Working Directory and GeoDatabase

In [9]:
# Set GDB Workspace
gdb_directory = r"/arcgis/home/data/gdb"
gdb_name = "bmsb_risk_analysis"
gdb_path = os.path.join(gdb_directory, f"{gdb_name}.gdb")

# Check if GDB Folder Exists
if not os.path.exists(gdb_directory):
    os.mkdir(gdb_directory)
    arcpy.management.CreateFileGDB(gdb_directory, gdb_name)
    
    print("GDB Created")

# Check if Directory and/or GDB Exist
else:
    if not os.path.exists(gdb_path):
        arcpy.management.CreateFileGDB(gdb_directory, gdb_name)
        
        print("Directory Exists and GDB Created")
        
    else:
        print("Directory and GDB Exist: ", gdb_path)

GDB Created


### 2. Set File Paths

In [25]:
# Set Paths for Downloaded Datasets
nlcd_tif = '/arcgis/home/data/mnNLCD/NLCD_2019_Land_Cover.tif'
cityboundaries_shp = '/arcgis/home/data/mnCityBoundaries/city_township_unorg.shp'
dem_gdb = '/arcgis/home/data/mnDEM/elev_30m_digital_elevation_model.gdb/digital_elevation_model_30m'
bmsbobservations_csv = '/arcgis/home/data/mnBMSBObservations/BMSBObservations.csv'

### 3. Extract, Transform, and Load RWIS Weather Data and BMSB Observation Data to GDB

In [21]:
# Extract and Transform RWIS Weather Data
weather_df = WeatherLoader.multi_month([4, 5, 6, 7, 8, 9], 2022)
weather_df # Display WX Values are the Mean Daily Values (i.e., Mean Maximum Temperture)

# Load RWIS Weather Data to GDB
WeatherLoader.load(gdb_path, "RWISWeather_2022", weather_df)

In [26]:
# Extract BMSB Observation Data 
bmsb_observations = ObservationLoader(bmsbobservations_csv)
bmsb_observations.df.head()

# Transform BMSB Observation Data 
bmsb_observations.transform()
bmsb_observations.df

# Load BMSB Observations Data to GDB
bmsb_observations.load(gdb_path, "BMSB_Observations")

/arcgis/home/functions/etl.py:248: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  self.df["ObsDate"] = self.df["ObsDate"].astype("datetime64[ns]")


### 4. Reclassify National Land Cover Dataset

In [23]:
# Reclassify National Land Cover Dataset to Urban, Agricultural, and Natural
reclassified_nlcd_tif = arcpy.sa.Reclassify(
    in_raster=nlcd_tif,
    reclass_field="NLCD_Land",
    remap="Unclassified NODATA;'Open Water' 3;'Developed, Open Space' 1;'Developed, Low Intensity' 1;'Developed, Medium Intensity' 1;'Developed, High Intensity' 1;'Barren Land' 3;'Deciduous Forest' 3;'Evergreen Forest' 3;'Mixed Forest' 3;Shrub/Scrub 3;Herbaceous 3;Hay/Pasture 2;'Cultivated Crops' 2;'Woody Wetlands' 3;'Emergent Herbaceous Wetlands' 3",
    missing_values="DATA"
)
reclassified_nlcd_tif.save(os.path.join(gdb_path, "Reclassified_NLCD_2019_Land_Cover"))


### 5. Load DEM and City Boundary Data to GDB

In [24]:
# Load DEM to GDB
arcpy.management.CopyRaster(dem_gdb, os.path.join(gdb_path, "Digital_Elevation_Model_30m"))

# Load City Boundary Data to GDB
arcpy.conversion.ExportFeatures(
    cityboundaries_shp,
    os.path.join(gdb_path, "City_Boundaries"),
    where_clause="CTU_CLASS = 'CITY'",
)

<Result '/arcgis/home/data/gdb/bmsb_risk_analysis.gdb/City_Boundaries'>

### 6. Aggregate Data

In [27]:
# Aggregate National Land Cover Dataset to City Boundaries
arcpy.sa.ZonalHistogram(
    os.path.join(gdb_path, "City_Boundaries"),
    "FEATURE_NA",
    os.path.join(gdb_path, "Reclassified_NLCD_2019_Land_Cover"),
    os.path.join(gdb_path, "Aggregated_NLCD_2019_Land_Cover"),
    zones_as_rows="ZONES_AS_ROWS"
)

<geoprocessing server result object at 0x7fb77ec71f60>

In [28]:
# Aggregate Digital Elevation Model (DEM) to City Boundaries
arcpy.ia.ZonalStatisticsAsTable(
    os.path.join(gdb_path, "City_Boundaries"),
    "FEATURE_NA",
    os.path.join(gdb_path, "Digital_Elevation_Model_30m"),
    os.path.join(gdb_path, "Aggregated_Digital_Elevation_Model_30m"),
    ignore_nodata="DATA",
    statistics_type="ALL"
)

<geoprocessing server result object at 0x7fb77ec71ed0>

In [29]:
# Aggregate BMSB Observation Data to City Boundaries
arcpy.analysis.SummarizeWithin(
    os.path.join(gdb_path, "City_Boundaries"),
    os.path.join(gdb_path, "BMSB_Observations"),
    os.path.join(gdb_path, "Aggregated_BMSB_Observations"),
    keep_all_polygons="KEEP_ALL",
    sum_fields=None,
    sum_shape="ADD_SHAPE_SUM",
    shape_unit="SQUAREKILOMETERS",
    group_field=None,
    add_min_maj="NO_MIN_MAJ",
    add_group_percent="NO_PERCENT",
    out_group_table=None
)

<Result '/arcgis/home/data/gdb/bmsb_risk_analysis.gdb/Aggregated_BMSB_Observations'>

### 7. Join Data

In [30]:
# Join Nearest Weather Data to City Boundary Data 
arcpy.analysis.SpatialJoin(
    os.path.join(gdb_path, "City_Boundaries"),
    os.path.join(gdb_path, "RWISWeather_2022"),
    os.path.join(gdb_path, "Aggregated_RWISWeather_2022"),
    join_operation="JOIN_ONE_TO_ONE",
    join_type="KEEP_ALL",
    match_option="CLOSEST",
    search_radius=None,
    distance_field_name=""
)

<Result '/arcgis/home/data/gdb/bmsb_risk_analysis.gdb/Aggregated_RWISWeather_2022'>

In [ ]:
# Join Aggregated Data to City Boundary Data 
joined = arcpy.management.AddJoin(os.path.join(gdb_path, "City_Boundaries"),"FEATURE_NA", os.path.join(gdb_path, "Aggregated_Digital_Elevation_Model_30m"),"FEATURE_NA")

joined = arcpy.management.AddJoin(
    joined,
    "FEATURE_NA",
    os.path.join(gdb_path, "Aggregated_RWISWeather_2022"),
    "FEATURE_NA"
)

joined = arcpy.management.AddJoin(
    joined,
    "FEATURE_NA",
    os.path.join(gdb_path, "Aggregated_BMSB_Observations"),
    "FEATURE_NA"
)

joined = arcpy.management.AddJoin(
    joined,
    "FEATURE_NA",
    os.path.join(gdb_path, "Aggregated_NLCD_2019_Land_Cover"),
    "LABEL"
)

# Copy Features from Joined Feature Class
arcpy.management.CopyFeatures(
    joined,
    os.path.join(gdb_path, "Aggregated_City_Boundaries")
)

In [35]:
# Display as SEDF
pd.DataFrame.spatial.from_featureclass(os.path.join(gdb_path, "Aggregated_City_Boundaries")).head()

,OBJECTID,City_Boundaries_GNIS_FEATU,City_Boundaries_FEATURE_NA,City_Boundaries_CTU_CLASS,City_Boundaries_COUNTY_GNI,City_Boundaries_COUNTY_COD,City_Boundaries_COUNTY_NAM,City_Boundaries_POPULATION,City_Boundaries_SHAPE_Leng,Aggregated_Digital_Elevation_Model_30m_OBJECTID,...,Aggregated_BMSB_Observations_COUNTY_NAM,Aggregated_BMSB_Observations_POPULATION,Aggregated_BMSB_Observations_SHAPE_Leng,Aggregated_BMSB_Observations_Point_Count,Aggregated_NLCD_2019_Land_Cover_OBJECTID,Aggregated_NLCD_2019_Land_Cover_LABEL,Aggregated_NLCD_2019_Land_Cover_CLASS_1,Aggregated_NLCD_2019_Land_Cover_CLASS_2,Aggregated_NLCD_2019_Land_Cover_CLASS_3,SHAPE
0,1,2394210,Bluffton,CITY,659501,56,Otter Tail,208,12870.280762,1,...,Otter Tail,208,12870.280762,0,1,Bluffton,794,4350,2728,"{""rings"": [[[329659.49899999984, 5147112.93899..."
1,2,2396539,Sartell,CITY,659517,73,Stearns,17750,90672.68153,2,...,Stearns,17750,90672.68153,0,2,Sartell,19594,7527,4141,"{""rings"": [[[401466.0499999998, 5050916.149], ..."
2,3,2393505,Cambridge,CITY,659475,30,Isanti,10541,65438.361367,3,...,Isanti,10541,65438.361367,1,3,Cambridge,12836,4015,5822,"{""rings"": [[[479383.5999999996, 5046640.25], [..."
3,4,2397203,Waseca,CITY,659525,81,Waseca,9306,31106.933645,4,...,Waseca,9306,31106.933645,0,4,Waseca,9359,1836,4548,"{""rings"": [[[456193.8666000003, 4880132.7445],..."
4,5,2395562,La Crescent,CITY,659473,28,Houston,5365,24285.037149,5,...,Houston,5365,24285.037149,1,5,La Crescent,5607,537,4860,"{""rings"": [[[633758.4579999996, 4853600.790999..."


### 8. Clean Data

In [39]:
# Clean Joined Dataset
arcpy.management.DeleteField(
    os.path.join(gdb_path, "Aggregated_City_Boundaries"),
    "City_Boundaries_GNIS_FEATU;City_Boundaries_FEATURE_NA;City_Boundaries_COUNTY_NAM;City_Boundaries_POPULATION;City_Boundaries_SHAPE_Leng;Aggregated_Digital_Elevation_Model_30m_RANGE;Aggregated_Digital_Elevation_Model_30m_MEAN;Aggregated_Digital_Elevation_Model_30m_STD;Aggregated_Digital_Elevation_Model_30m_MEDIAN;Aggregated_RWISWeather_2022_Join_Count;Aggregated_RWISWeather_2022_max_tmpf;Aggregated_RWISWeather_2022_min_tmpf;Aggregated_RWISWeather_2022_precip;Aggregated_BMSB_Observations_Point_Count;Aggregated_NLCD_2019_Land_Cover_CLASS_1;Aggregated_NLCD_2019_Land_Cover_CLASS_2;Aggregated_NLCD_2019_Land_Cover_CLASS_3",
    "KEEP_FIELDS"
)

# Display as SEDF
pd.DataFrame.spatial.from_featureclass(os.path.join(gdb_path, "Aggregated_City_Boundaries")).head()

,OBJECTID,City_Boundaries_GNIS_FEATU,City_Boundaries_FEATURE_NA,City_Boundaries_COUNTY_NAM,City_Boundaries_POPULATION,City_Boundaries_SHAPE_Leng,Aggregated_Digital_Elevation_Model_30m_RANGE,Aggregated_Digital_Elevation_Model_30m_MEAN,Aggregated_Digital_Elevation_Model_30m_STD,Aggregated_Digital_Elevation_Model_30m_MEDIAN,Aggregated_RWISWeather_2022_Join_Count,Aggregated_RWISWeather_2022_max_tmpf,Aggregated_RWISWeather_2022_min_tmpf,Aggregated_RWISWeather_2022_precip,Aggregated_BMSB_Observations_Point_Count,Aggregated_NLCD_2019_Land_Cover_CLASS_1,Aggregated_NLCD_2019_Land_Cover_CLASS_2,Aggregated_NLCD_2019_Land_Cover_CLASS_3,SHAPE
0,1,2394210,Bluffton,Otter Tail,208,12870.280762,84,1347.493968,21.954257,1353,1,69.002294,49.479673,0.000164,0,794,4350,2728,"{""rings"": [[[329659.49899999984, 5147112.93899..."
1,2,2396539,Sartell,Stearns,17750,90672.68153,120,1047.734411,16.158058,1047,1,69.840329,50.023607,0.000492,0,19594,7527,4141,"{""rings"": [[[401466.0499999998, 5050916.149], ..."
2,3,2393505,Cambridge,Isanti,10541,65438.361367,85,949.380606,17.984951,955,1,70.462952,49.659673,0.000273,1,12836,4015,5822,"{""rings"": [[[479383.5999999996, 5046640.25], [..."
3,4,2397203,Waseca,Waseca,9306,31106.933645,56,1136.962191,12.658397,1135,1,72.184263,52.943936,0.000383,0,9359,1836,4548,"{""rings"": [[[456193.8666000003, 4880132.7445],..."
4,5,2395562,La Crescent,Houston,5365,24285.037149,584,725.380685,128.775186,679,1,74.276315,58.962108,0.0,1,5607,537,4860,"{""rings"": [[[633758.4579999996, 4853600.790999..."


In [110]:
# Convert to SEDF
df = pd.DataFrame.spatial.from_featureclass(os.path.join(gdb_path, "Aggregated_City_Boundaries"))

In [111]:
df = df.rename(columns={
    'OBJECTID': 'ObjectID',
    'City_Boundaries_GNIS_FEATU': 'GNIS',
    'City_Boundaries_FEATURE_NA': 'City',
    'City_Boundaries_COUNTY_NAM': 'County',
    'City_Boundaries_POPULATION': 'Population',
    'City_Boundaries_SHAPE_Leng': 'Shape_Length',
    
    'Aggregated_Digital_Elevation_Model_30m_RANGE': 'DEM: Range',
    'Aggregated_Digital_Elevation_Model_30m_MEAN': 'DEM: Mean',
    'Aggregated_Digital_Elevation_Model_30m_STD': 'DEM: Standard Deviation',
    'Aggregated_Digital_Elevation_Model_30m_MEDIAN': 'DEM: Median',
    
    'Aggregated_RWISWeather_2022_Join_Count': 'WX: Join Count',
    'Aggregated_RWISWeather_2022_max_tmpf': 'WX: Mean Maximum Temperature',
    'Aggregated_RWISWeather_2022_min_tmpf': 'WX: Mean Minimum Temperature',
    'Aggregated_RWISWeather_2022_precip': 'WX: Mean Precipitation',
    'Aggregated_BMSB_Observations_Point_Count': 'BMSB: Observation Count',
    
    'Aggregated_NLCD_2019_Land_Cover_CLASS_1': 'NLCD: Urban',
    'Aggregated_NLCD_2019_Land_Cover_CLASS_2': 'NLCD: Agricultral',
    'Aggregated_NLCD_2019_Land_Cover_CLASS_3': 'NLCD: Natural',

})


In [112]:
# Convert NLCD Classifications to Percentages
df["NLCD: Percent Urban"] = df["NLCD: Urban"] / (df["NLCD: Urban"] + df["NLCD: Agricultral"] + df["NLCD: Natural"])
df["NLCD: Percent Agricultral"] = df["NLCD: Agricultral"] / (df["NLCD: Urban"] + df["NLCD: Agricultral"] + df["NLCD: Natural"])
df["NLCD: Percent Natural"] = df["NLCD: Natural"] / (df["NLCD: Urban"] + df["NLCD: Agricultral"] + df["NLCD: Natural"])

In [113]:
# Convert BMSB Observation Count to Presence
df["BMSB: Presence"] = df["BMSB: Observation Count"].apply(lambda x: 1 if x >= 1 else 0)

In [114]:
# Drop Columns
df = df.drop(['Shape_Length', 'County', 'GNIS', 'WX: Join Count', 'NLCD: Urban', 'NLCD: Agricultral', 'NLCD: Natural', 'BMSB: Observation Count'], axis=1)

In [108]:
df.columns = ['ObjectID', 'City', 'Population',
       'DEM: Range', 'DEM: Mean', 'DEM: Standard Deviation',
       'DEM: Median', 
       'WX: Mean Maximum Temperature', 'WX: Mean Minimum Temperature', 'WX: Mean Precipitation',
       'BMSB: Presence', 'NLCD: Percent Urban',
       'NLCD: Percent Agricultral', 'NLCD: Percent Natural', 'SHAPE'
]